In [1]:
import re
import json

import pandas as pd
from notion.block import CollectionViewBlock

import helpers

In [58]:
from imp import reload
reload(helpers)

<module 'helpers' from '/Users/jrw/epidemics/containment/helpers.py'>

In [2]:
MEASURES_DB_URL = 'https://www.notion.so/977d5e5be0434bf996704ec361ad621d?v=fe54f89ca9e04ac799af42b39e1efc4b'
COMPLETENESS_DB_URL = 'https://www.notion.so/dbf0f4ffc03f4417a16d65a6d08eea69?v=845ef4baafd44e72b9c483381b90caae'

In [3]:
client = helpers.get_notion_client()

In [88]:
tags = pd.read_csv('data/tags.csv')

In [4]:
measures_db = client.get_block(MEASURES_DB_URL)
completeness_db = client.get_block(COMPLETENESS_DB_URL)

In [ ]:
# Notion API seems to take a long time with large tables
measures = helpers.df_from_notion_table(measures_db)

In [59]:
completeness = helpers.df_from_notion_table(
    completeness_db)[['country', 'complete_up_to_date']].dropna()

In [93]:
md = measures.dropna()

In [68]:
cm = measures.merge(completeness, on='country')

In [106]:
cm = cm[cm.keywords.apply(len) > 0]

In [108]:
cm.keywords.apply(len).sum()

1967

In [109]:
len(cm)

1288

In [101]:
cm.loc[262]

target_region                                                                        []
source                                https://www.hongkongfp.com/2020/03/21/breaking...
keywords                              [public mask and hygiene supply, social distan...
country                                                                       Hong Kong
date_end_intended                                                   2020-04-24 00:00:00
implementing_state_province                                                          []
target_state                                                                           
implementing_city_municipality                                                       []
target_city                                                                            
applies_to                                                                             
date_start                                                          2020-03-21 00:00:00
exceptions                      

In [145]:
cm.implementing_city_municipality.apply(lambda x: None if len(x) == 0 else x).dropna()

5             [Ischgl]
89                  []
105            [Daegu]
106            [gunpo]
108            [Daegu]
             ...      
1249                []
1255                []
1296          [Bansko]
1302           [Sofia]
1321    [Vatican City]
Name: implementing_city_municipality, Length: 105, dtype: object

In [149]:
fields_to_use, renamed_fields = (list(cols) for cols in zip(
#     ('id', 'id'), # not reliable
    ('country', 'country'),
#     # maybe worth adding these:
#     ('implementing_state_province', 'state_or_province'),
#     ('implementing_city_municipality', 'municipality'),
#     ('target_country', 'target_country'),
    ('description_of_measure_implemented', 'description'),
    ('source', 'source'),
    ('date_start', 'start_date'),
    ('date_end_intended', 'end_date'),
    ('quantity', 'quantity'),
    ('keywords', 'tags')
))

# other columns were not include b/c too few rows have
# data to bother programming them into the frontend

In [150]:
mc2 = cm[fields_to_use].copy()
mc2.columns = renamed_fields

In [152]:
def expand_tags(measures):
    tagged_rows = []
    for _, measure in measures.iterrows():
        for tag in measure['tags']:
            row = dict(tag=tag, **measure.to_dict())
            del row['tags']
            tagged_rows.append(row)
    return pd.DataFrame(tagged_rows)

In [154]:
tagged = expand_tags(mc2)

In [156]:
tags.aggregation.unique()

array(['max', 'sum unique', None, 'min', 'check notes'], dtype=object)

In [214]:
def get_default_quantities():
    quant_map = tags[['tag', 'quantity']].drop_duplicates(subset=['tag'], keep='last')
    quantities = quant_map.quantity[
        quant_map.quantity.str.match(r'[\d.]+').fillna(False)].astype('f')
    quant_map['quantity'] = None
    quant_map.quantity.fillna(quantities, inplace=True)
    return quant_map

In [215]:
quantified = tagged.merge(get_default_quantities(), on='tag', suffixes=('_measure', ''))

quantified.quantity.fillna(quantified.quantity_measure, inplace=True)
quantified.drop(columns=['quantity_measure'], inplace=True)

In [222]:
quantified[quantified.description.str.match(r'\b[Xx]\b')]

,tag,country,description,source,start_date,end_date,quantity


In [228]:
def measures_to_api_object(measures):
    return {**helpers.get_api_metadata(client),
            'data': {'containment_measures': list(measures.T.to_dict().values())}}